# 教务系统 RAG + Text2SQL 性能优化指南

作为一名高级全栈AI开发专家，本 Notebook 将指导你完成对一个基于 **MySQL** 和 **多文件 RAG** 的教务系统的性能分析与优化。我们将从数据库查询、索引策略，到 RAG 管道中的数据加载、分块、嵌入、检索等环节，进行全面的诊断和改进。

## 1. 环境设置与数据库连接

在开始之前，请确保你已安装所有必要的库。我们将使用 `pymysql` 连接数据库，`SQLAlchemy` 作为 ORM，`pandas` 进行数据处理，以及 `langchain` 作为 RAG 框架。

**任务**:
- 导入所需库。
- 从环境变量或配置文件中安全地加载数据库凭据。
- 创建一个 SQLAlchemy 引擎，并测试与教务系统 MySQL 数据库的连接。

In [1]:
import os
import pandas as pd
from sqlalchemy import create_engine, text
from dotenv import load_dotenv

# 加载 .env 文件，如果您的凭据存储在其中
load_dotenv()

# --- 从环境变量或直接在此处配置数据库连接 ---
# 参考您的 '我的汇总备份.ipynb'，我们使用 pymysql 连接
# 请确保您的 .env 文件或环境变量中包含以下信息
DB_USER = os.getenv("DB_USER", "root")  # 您的 MySQL 用户名
DB_PASSWORD = os.getenv("DB_PASSWORD", "Root123!") # 您的 MySQL 密码（已更新为您的实际密码）
DB_HOST = os.getenv("DB_HOST", "localhost") # 数据库主机，通常是 localhost
DB_PORT = os.getenv("DB_PORT", "3306") # 数据库端口，默认 3306
DB_NAME = os.getenv("DB_NAME", "college_db") # 目标数据库名称

# 创建 SQLAlchemy 连接字符串
# 我们使用 'mysql+pymysql' 作为连接器
DATABASE_URL = f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

try:
    # 创建 SQLAlchemy 引擎
    engine = create_engine(DATABASE_URL)

    # 测试连接
    with engine.connect() as connection:
        print("✅ 数据库连接成功！")
        # 简单查询，验证连接有效
        result = connection.execute(text("SELECT 1"))
        print(" |--- 测试查询 (SELECT 1) 返回:", result.scalar())
except Exception as e:
    print(f"❌ 数据库连接失败，请检查你的配置: {e}")
    print("   请确保 'pymysql' 和 'SQLAlchemy' 已安装: pip install pymysql sqlalchemy")
    engine = None

✅ 数据库连接成功！
 |--- 测试查询 (SELECT 1) 返回: 1


## 2. MySQL 性能分析与诊断

性能优化的第一步是识别瓶颈。对于 Text2SQL 系统，最常见的瓶颈是数据库端的慢查询。我们将使用 `EXPLAIN` 命令来分析那些由 RAG 系统生成的、或与 RAG 相关的 SQL 查询。

**任务**:
- 识别一个或多个在 RAG 流程中常用但性能可能不佳的查询。
- 使用 `EXPLAIN` 分析这些查询的执行计划。
- 重点关注 `type`（是否为 `ALL`，即全表扫描）、`key`（是否使用了索引）和 `Extra`（是否有 `Using filesort` 或 `Using temporary`）等字段。

In [2]:
def analyze_query(sql_query: str):
    """使用 EXPLAIN 分析给定的 SQL 查询"""
    if not engine:
        print("数据库未连接，无法分析查询。")
        return

    with engine.connect() as connection:
        # 在查询前加上 EXPLAIN
        explain_query = f"EXPLAIN {sql_query}"
        try:
            result = connection.execute(text(explain_query))
            df = pd.DataFrame(result.fetchall(), columns=result.keys())
            print(f"🔍 对以下查询的 EXPLAIN 分析结果:")
            print(f"   '{sql_query}'")
            display(df)
        except Exception as e:
            print(f"❌ 分析查询时出错: {e}")

# --- 示例：分析一个可能低效的查询 ---
# 假设我们要根据课程标题的关键词和系名来查找课程
# 这个查询在 `course.title` 和 `department.dept_name` 上没有索引时可能会很慢
slow_query_example = """
SELECT c.title, c.credits, d.dept_name
FROM course AS c
JOIN department AS d ON c.dept_name = d.dept_name
WHERE c.title LIKE '%Introduction%' AND d.building = 'Watson';
"""

analyze_query(slow_query_example)

🔍 对以下查询的 EXPLAIN 分析结果:
   '
SELECT c.title, c.credits, d.dept_name
FROM course AS c
JOIN department AS d ON c.dept_name = d.dept_name
WHERE c.title LIKE '%Introduction%' AND d.building = 'Watson';
'


,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,SIMPLE,d,None,ref,"PRIMARY,idx_department_building",idx_department_building,63,const,1,100.00,Using index
1,1,SIMPLE,c,None,ref,dept_name,dept_name,83,college_db.d.dept_name,10,11.11,Using where


## 3. 查询优化与索引策略

根据 `EXPLAIN` 的分析结果，我们可以采取措施优化查询。最直接有效的方法是为查询中 `WHERE` 子句和 `JOIN` 操作涉及的列创建索引。

**任务**:
- 识别 `EXPLAIN` 结果中未使用索引的 `WHERE` 或 `JOIN` 列。
- 编写 `CREATE INDEX` 语句为这些列添加索引。
- 重新运行 `EXPLAIN`，验证新索引是否被查询优化器使用（`key` 字段应显示索引名称）。

In [3]:
def apply_and_verify_index(index_creation_sql: str, query_to_re_analyze: str):
    """应用索引并重新分析查询以验证效果"""
    if not engine:
        print("数据库未连接，无法执行操作。")
        return

    with engine.connect() as connection:
        try:
            # 开始一个事务
            trans = connection.begin()
            print(f"🚀 正在尝试创建索引: {index_creation_sql}")
            connection.execute(text(index_creation_sql))
            trans.commit()
            print("✅ 索引创建成功（或已存在）。")
        except Exception as e:
            print(f"⚠️ 创建索引时出现问题 (可能是索引已存在): {e}")
            if 'trans' in locals() and trans.is_active:
                trans.rollback()

    # 再次分析查询
    print("\n--- 重新分析查询执行计划 ---")
    analyze_query(query_to_re_analyze)

# --- 示例：为 `course.title` 和 `department.building` 创建索引 ---
# 注意：对于 LIKE '%...%' 查询，B-Tree 索引效果有限，但对于前缀匹配或全文索引则非常有效。
# 这里我们仍然创建一个常规索引作为示例。
index_sql_1 = "CREATE INDEX idx_course_title ON course(title);"
index_sql_2 = "CREATE INDEX idx_department_building ON department(building);"

# 为 course.title 创建索引并验证
apply_and_verify_index(index_sql_1, slow_query_example)

# 为 department.building 创建索引并验证
apply_and_verify_index(index_sql_2, slow_query_example)

🚀 正在尝试创建索引: CREATE INDEX idx_course_title ON course(title);
⚠️ 创建索引时出现问题 (可能是索引已存在): (pymysql.err.OperationalError) (1061, "Duplicate key name 'idx_course_title'")
[SQL: CREATE INDEX idx_course_title ON course(title);]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

--- 重新分析查询执行计划 ---
🔍 对以下查询的 EXPLAIN 分析结果:
   '
SELECT c.title, c.credits, d.dept_name
FROM course AS c
JOIN department AS d ON c.dept_name = d.dept_name
WHERE c.title LIKE '%Introduction%' AND d.building = 'Watson';
'


,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,SIMPLE,d,None,ref,"PRIMARY,idx_department_building",idx_department_building,63,const,1,100.00,Using index
1,1,SIMPLE,c,None,ref,dept_name,dept_name,83,college_db.d.dept_name,10,11.11,Using where


🚀 正在尝试创建索引: CREATE INDEX idx_department_building ON department(building);
⚠️ 创建索引时出现问题 (可能是索引已存在): (pymysql.err.OperationalError) (1061, "Duplicate key name 'idx_department_building'")
[SQL: CREATE INDEX idx_department_building ON department(building);]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

--- 重新分析查询执行计划 ---
🔍 对以下查询的 EXPLAIN 分析结果:
   '
SELECT c.title, c.credits, d.dept_name
FROM course AS c
JOIN department AS d ON c.dept_name = d.dept_name
WHERE c.title LIKE '%Introduction%' AND d.building = 'Watson';
'


,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,SIMPLE,d,None,ref,"PRIMARY,idx_department_building",idx_department_building,63,const,1,100.00,Using index
1,1,SIMPLE,c,None,ref,dept_name,dept_name,83,college_db.d.dept_name,10,11.11,Using where


## 4. RAG 管道分析：数据加载与分块

RAG 的性能不仅取决于数据库，还严重依赖于输入文档的质量和处理方式。我们将首先分析数据加载和文本分块（Chunking）策略。

**任务**:
- 使用 `DirectoryLoader` (from `langchain`) 或类似工具加载你的多文件知识库（例如，教学大纲、学生手册等）。
- 评估当前的文本分块策略。一个好的分块策略应在保持语义完整性和限制单块大小之间取得平衡。
- 试验不同的 `chunk_size` 和 `chunk_overlap` 值，观察它们如何影响检索上下文的质量。

In [4]:
# 确保安装了 langchain
# !pip install langchain langchain-community langchain-text-splitters

from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# --- 1. 准备并加载文档 ---
# 目标目录
DOCS_PATH = "knowledge_base/docs"
NUM_FILES_TO_GENERATE = 100

# 自动生成示例文件（如果目录不存在或文件不足）
if not os.path.exists(DOCS_PATH) or len([f for f in os.listdir(DOCS_PATH) if f.endswith('.txt')]) < NUM_FILES_TO_GENERATE:
    os.makedirs(DOCS_PATH, exist_ok=True)
    print(f"目录 '{DOCS_PATH}' 不存在或文件不足，正在生成 {NUM_FILES_TO_GENERATE} 个示例文档...")
    for i in range(NUM_FILES_TO_GENERATE):
        file_name = os.path.join(DOCS_PATH, f"document_{i+1}.txt")
        with open(file_name, "w", encoding="utf-8") as f:
            if i == 0:
                f.write("课程: CS101, 计算机科学导论. 授课教师: Alan Turing. 本课程涵盖算法、数据结构和计算理论的基础知识。")
            elif i == 1:
                f.write("教学政策: 所有学生必须遵守学术诚信原则，严禁抄袭。关于选课和退课的详细规定请参考学生手册第5章。")
            else:
                f.write(f"这是第 {i+1} 号教学文档的占位内容。这里可以包含任何关于课程、政策、或校园生活的文本信息。")
    print("✅ 示例文档生成完毕。")

# --- 2. 批量加载文档 ---
try:
    loader = DirectoryLoader(DOCS_PATH, glob="**/*.txt", loader_cls=TextLoader, loader_kwargs={'encoding': 'utf-8'})
    documents = loader.load()
    print(f"✅ 成功从 '{DOCS_PATH}' 批量加载了 {len(documents)} 个文档。")
except Exception as e:
    print(f"❌ 加载文档失败: {e}")
    documents = []

# --- 3. 试验不同的分块策略 ---
if documents:
    text_sample = documents[0].page_content

    # 策略 1: 较小的块，较大的重叠
    splitter_small = RecursiveCharacterTextSplitter(chunk_size=128, chunk_overlap=20)
    chunks_small = splitter_small.split_text(text_sample)
    print(f"\n--- 策略 1 (chunk_size=128, overlap=20) ---")
    print(f"生成了 {len(chunks_small)} 个块。")

    # 策略 2: 较大的块，适中的重叠
    splitter_large = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=50)
    chunks_large = splitter_large.split_text(text_sample)
    print(f"\n--- 策略 2 (chunk_size=512, overlap=50) ---")
    print(f"生成了 {len(chunks_large)} 个块。")

✅ 成功从 'knowledge_base/docs' 批量加载了 100 个文档。

--- 策略 1 (chunk_size=128, overlap=20) ---
生成了 1 个块。

--- 策略 2 (chunk_size=512, overlap=50) ---
生成了 1 个块。


## 5. RAG 管道分析：嵌入与向量存储 (LangChain + ChromaDB)

嵌入模型（Embedding Model）的质量和向量数据库的效率直接影响检索的准确性和速度。

**任务**:
- 使用 LangChain 和 ChromaDB 将文档切片并向量化入库。
- 同时在 MySQL 中创建索引表，记录文件名与向量 ID 的对应关系。

In [ ]:
# 确保安装了所需的库
# !pip install langchain langchain-community langchain-text-splitters chromadb dashscope

import os
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import DashScopeEmbeddings
import uuid

# --- 1. 配置嵌入模型 (使用 DashScope API，避免下载 HuggingFace 模型) ---
# ⚠️ 请在环境变量中设置 DASHSCOPE_API_KEY，或在 backend/.env 文件中配置
DASHSCOPE_API_KEY = os.getenv("DASHSCOPE_API_KEY", None)
if not DASHSCOPE_API_KEY:
    print("⚠️ 警告：未设置 DASHSCOPE_API_KEY 环境变量！")
    print("   请运行: export DASHSCOPE_API_KEY='your-api-key-here'")
    print("   或在 backend/.env 文件中添加: DASHSCOPE_API_KEY=your-api-key-here")

try:
    # 使用环境变量中的 DashScope API Key
    embed_model = DashScopeEmbeddings(
        model="text-embedding-v2",
        dashscope_api_key=DASHSCOPE_API_KEY
    )
    # 测试嵌入模型是否工作
    test_embedding = embed_model.embed_query("测试文本")
    print(f"✅ 使用 DashScope 嵌入模型: 'text-embedding-v2'")
    print(f"   嵌入向量维度: {len(test_embedding)}")
except Exception as e:
    print(f"❌ 加载 DashScope 嵌入模型失败。错误: {e}")
    print("   请确保已安装 dashscope: pip install dashscope")
    embed_model = None

# --- 2. 切分文档 ---
if documents and embed_model:
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=256, chunk_overlap=30)
    docs = text_splitter.split_documents(documents)
    print(f"✅ 文档切分完成，共生成 {len(docs)} 个文本块。")

    # --- 3. 向量化并存入 ChromaDB，同时在 MySQL 中记录索引 ---
    persist_directory = 'chroma_db_langchain'
    
    # 为每个文档块生成唯一的ID
    ids = [str(uuid.uuid4()) for _ in docs]
    
    # 使用 from_documents 一步完成嵌入和存储
    vectordb = Chroma.from_documents(
        documents=docs, 
        embedding=embed_model,
        ids=ids,
        persist_directory=persist_directory
    )
    print(f"✅ 文本块已向量化并存入 ChromaDB。数据库位置: '{os.path.abspath(persist_directory)}'")

    # --- 4. 将文件名和向量ID的映射关系存入 MySQL ---
    if engine:
        # 先创建表
        try:
            with engine.connect() as connection:
                create_table_sql = text("""
                CREATE TABLE IF NOT EXISTS file_vector_index (
                    id INT AUTO_INCREMENT PRIMARY KEY,
                    file_name VARCHAR(255) NOT NULL,
                    vector_id VARCHAR(255) NOT NULL,
                    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                    INDEX idx_file_name (file_name),
                    INDEX idx_vector_id (vector_id)
                );
                """)
                connection.execute(create_table_sql)
                connection.commit()
                print("✅ 文件-向量索引表 'file_vector_index' 已成功创建或已存在。")
        except Exception as e:
            print(f"❌ 创建 'file_vector_index' 表失败: {e}")

        # 写入映射数据
        mappings = []
        for i, doc in enumerate(docs):
            file_name = doc.metadata.get('source', 'unknown')
            vector_id = ids[i]
            mappings.append({'file_name': file_name, 'vector_id': vector_id})
        
        if mappings:
            try:
                with engine.connect() as connection:
                    connection.execute(text("INSERT INTO file_vector_index (file_name, vector_id) VALUES (:file_name, :vector_id)"), mappings)
                    connection.commit()
                    print(f"✅ 成功将 {len(mappings)} 条文件-向量映射关系写入 MySQL 'file_vector_index' 表。")
            except Exception as e:
                print(f"❌ 写入 MySQL 索引表失败: {e}")
    
    # 定义一个 retriever 以便后续步骤使用
    retriever = vectordb.as_retriever(search_kwargs={"k": 5})
    print("✅ 检索器 (Retriever) 已准备就绪。")

else:
    print("⚠️ 文档或嵌入模型未准备好，跳过向量化入库。")
    vectordb = None
    retriever = None

✅ 使用 DashScope 嵌入模型: 'text-embedding-v2'
   嵌入向量维度: 1536
✅ 文档切分完成，共生成 100 个文本块。
✅ 文本块已向量化并存入 ChromaDB。数据库位置: '/home/wangct/homework/text2sql/sqlmodel/chroma_db_langchain'
✅ 文件-向量索引表 'file_vector_index' 已成功创建或已存在。
✅ 成功将 100 条文件-向量映射关系写入 MySQL 'file_vector_index' 表。
✅ 检索器 (Retriever) 已准备就绪。


## 6. 混合搜索：实现 Text2SQL + RAG

现在，我们将整合所有部分，创建一个强大的混合搜索函数 `hybrid_ask`。当用户提问时，此函数将：
1.  **并行执行 Text2SQL**: 将用户问题转换为 SQL 查询，从 MySQL 数据库中获取结构化数据。
2.  **并行执行向量检索**: 从 ChromaDB 中检索与问题相关的非结构化文档（教学政策、课程大纲等）。
3.  **LLM 综合回答**: 将上述两部分结果汇总，交给一个大型语言模型（LLM）来生成最终的、全面的答案。

In [ ]:
# --- 0. 检查 LLM 客户端 ---
# ⚠️ 请在环境变量中设置 DASHSCOPE_API_KEY，或在 backend/.env 文件中配置
# 我们将使用环境变量中的 API Key 来初始化 client
if 'client' not in locals() or client is None:
    try:
        from openai import OpenAI
        
        # 从环境变量获取 API Key
        DASHSCOPE_API_KEY = os.getenv("DASHSCOPE_API_KEY", None)
        if not DASHSCOPE_API_KEY:
            print("⚠️ 警告：未设置 DASHSCOPE_API_KEY 环境变量！")
            print("   请运行: export DASHSCOPE_API_KEY='your-api-key-here'")
            print("   或在 backend/.env 文件中添加: DASHSCOPE_API_KEY=your-api-key-here")
        
        client = OpenAI(
            api_key=DASHSCOPE_API_KEY,
            base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"
        )
        print("✅ 已根据您的风格初始化 DashScope client。")
    except Exception as e:
        print(f"❌ 初始化 DashScope client 失败: {e}")
        client = None
else:
    print("✅ 复用已有的 DashScope client。")

# --- 配置使用的模型 ---
# qwen-plus 免费额度用完后，可以改用 qwen-turbo（更便宜/免费额度更多）
LLM_MODEL = "qwen-turbo"  # 可选: "qwen-plus", "qwen-turbo", "qwen-max"

# --- 1. 创建 Text2SQL 工具 (风格对齐) ---
def get_schema(engine):
    """获取数据库的 schema 信息"""
    try:
        with open("backend/college_db_schema.txt", "r") as f:
            return f.read()
    except FileNotFoundError:
        return "错误：找不到 schema 文件。"

def run_sql(query: str):
    """执行 SQL 查询并返回结果（格式化为更易读的形式）"""
    if not engine:
        return "数据库未连接。"
    try:
        # 清理 SQL 查询，移除可能的 markdown 代码块标记和注释
        clean_query = query.strip()
        if clean_query.startswith("```sql"):
            clean_query = clean_query[6:]
        if clean_query.startswith("```"):
            clean_query = clean_query[3:]
        if clean_query.endswith("```"):
            clean_query = clean_query[:-3]
        clean_query = clean_query.strip()
        
        # 移除 SQL 注释（-- 开头的行）
        lines = clean_query.split('\n')
        clean_lines = [line for line in lines if not line.strip().startswith('--')]
        clean_query = '\n'.join(clean_lines).strip()
        
        # 如果清理后为空，返回提示
        if not clean_query:
            return "无有效的 SQL 查询。"
        
        with engine.connect() as connection:
            df = pd.read_sql(text(clean_query), connection)
            
            # 改进输出格式，使 LLM 更容易理解
            if df.empty:
                return "查询结果为空，数据库中没有匹配的记录。"
            
            # 格式化输出：明确说明查询到了多少条记录
            result_lines = [f"查询成功！共找到 {len(df)} 条记录："]
            result_lines.append(df.to_string(index=False))
            return "\n".join(result_lines)
    except Exception as e:
        return f"SQL 执行错误: {e}"

def generate_sql_from_question(question: str, db_schema: str):
    """使用您的 client 风格，将问题转换为 SQL"""
    prompt = f"""
    根据下面的数据库表结构，将用户的问题转换为一个 SQL 查询。
    
    重要规则：
    1. 只返回 SQL 查询语句，不要添加任何其他解释或注释
    2. 如果问题包含多个部分，只处理能用 SQL 查询的部分
    3. 注意表之间的正确关联关系：
       - instructor 通过 teaches 表与 course 关联
       - teaches 表包含 (ID, course_id, sec_id, semester, year)
       - course 表包含 (course_id, title, dept_name, credits)
       - instructor 表包含 (ID, name, dept_name, salary)

    数据库表结构:
    {db_schema}

    用户问题: {question}

    SQL 查询:
    """
    response = client.chat.completions.create(
        model=LLM_MODEL,
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

# --- 2. 实现 hybrid_ask 函数 (风格对齐) ---
def hybrid_ask(question: str):
    if not client or not retriever or not engine:
        print("⚠️ 系统未完全初始化 (Client, Retriever, 或 Engine 未准备好)。")
        return

    # -- 步骤 A: 并行获取上下文 --
    
    # A.1: RAG 检索非结构化文档
    print("🔍 正在进行向量检索...")
    rag_docs = retriever.invoke(question)
    rag_context = "\n\n".join([doc.page_content for doc in rag_docs])
    print("  - RAG 上下文获取完毕。")

    # A.2: Text2SQL 获取结构化数据
    print("🔍 正在生成并执行 SQL 查询...")
    db_schema = get_schema(engine)
    generated_sql = generate_sql_from_question(question, db_schema)
    print(f"  - 生成的 SQL: \n{generated_sql}")
    sql_results = run_sql(generated_sql)
    print(f"  - SQL 结果: \n{sql_results}")

    # -- 步骤 B: LLM 综合回答 --
    # 改进提示词，明确告诉 LLM 如何使用数据库查询结果
    final_prompt = f"""
    你是一个专业的教务系统问答助手。请根据以下提供的上下文信息，全面而准确地回答用户的问题。

    **用户问题:**
    {question}

    ---
    **相关文档信息 (来自教学政策、课程大纲等):**
    {rag_context}
    ---
    **数据库查询结果 (这是从教务数据库中查询到的真实数据，请优先使用这些数据回答问题):**
    {sql_results}
    ---

    **回答要求:**
    1. 如果数据库查询结果包含有效数据，请直接使用这些数据回答问题
    2. 如果数据库查询结果为空或出错，再考虑从文档信息中寻找答案
    3. 回答要简洁、准确，直接给出查询到的具体数值或信息

    **你的综合回答:**
    """
    
    print("\n🤖 正在综合所有信息生成最终答案...")
    final_response = client.chat.completions.create(
        model=LLM_MODEL,
        messages=[{"role": "user", "content": final_prompt}]
    )
    final_answer = final_response.choices[0].message.content
    
    return final_answer

# --- 3. 运行示例 ---
if client:
    print("\n" + "="*30)
    print(f"混合搜索示例 (使用模型: {LLM_MODEL})")
    print("="*30 + "\n")
    
    # 示例问题，这个问题可能需要结合数据库和文档才能完美回答
    example_question = "查询主讲老师为 'Lembr' 的课程的学分是多少？"
    
    final_answer = hybrid_ask(example_question)
    
    print("\n--- 最终答案 ---")
    print(final_answer)

✅ 复用已有的 DashScope client。

混合搜索示例 (使用模型: qwen-turbo)

🔍 正在进行向量检索...
  - RAG 上下文获取完毕。
🔍 正在生成并执行 SQL 查询...
  - 生成的 SQL: 
SELECT course.credits  
FROM course  
JOIN teaches ON course.course_id = teaches.course_id  
JOIN instructor ON teaches.ID = instructor.ID  
WHERE instructor.name = 'Lembr';
  - SQL 结果: 
查询成功！共找到 2 条记录：
 credits
     4.0
     4.0

🤖 正在综合所有信息生成最终答案...

--- 最终答案 ---
查询到主讲老师为 'Lembr' 的课程的学分为 4.0。


## 7. 端到端性能基准测试

最后，为了量化我们的优化效果，我们需要建立一个简单的基准测试框架。

**任务**:
- **创建评估集**: 准备一个包含多个代表性问题和其"黄金标准"答案（或期望包含的关键词）的评估集。
- **定义评估指标**:
    - **响应时间**: 从提问到获得最终答案的总时间。
    - **检索准确率 (Hit Rate)**: 检索到的文档是否包含了生成正确答案所需的信息。
    - **答案质量**: （主观或使用 LLM-as-a-judge）生成的答案是否准确、完整。
- **执行基准测试**: 分别在优化前和优化后的系统上运行评估集，并记录各项指标，进行对比。

In [15]:
import time

# --- 1. 定义评估集 (使用数据库中真实存在的数据) ---
# 基于数据库实际数据：
# - 教师: Lembr, Bawa, Yazdi, Wieland, Romero, Mingoz, Dale 等
# - 课程: Image Processing, Manufacturing, Elastic Structures, Accounting 等
# - 关联: Romero 教 Image Processing, Mingoz 教 Manufacturing 等

eval_questions = [
    {
        "question": "查询主讲老师为 'Romero' 的课程名称是什么？",
        "expected_keywords": ["Image Processing"]
    },
    {
        "question": "Manufacturing 这门课程的授课老师是谁？",
        "expected_keywords": ["Mingoz"]
    },
    {
        "question": "关于学术诚信有什么规定？",
        "expected_keywords": ["学术诚信", "抄袭"]
    },
    {
        "question": "查询 'Lembr' 老师教授的课程学分是多少？",
        "expected_keywords": ["4"]
    },
]

# --- 2. 定义评估函数 ---
def run_benchmark(questions):
    total_time = 0
    hit_count = 0
    
    results = []

    for item in questions:
        q = item["question"]
        
        start_time = time.time()
        response = hybrid_ask(q)
        end_time = time.time()
        
        latency = end_time - start_time
        total_time += latency
        
        # 检查关键词是否在响应中
        response_text = str(response) if response else ""
        
        is_hit = all(keyword in response_text for keyword in item["expected_keywords"])
        if is_hit:
            hit_count += 1
            
        results.append({
            "question": q,
            "latency": latency,
            "is_hit": is_hit,
            "response": response_text[:200] + "..." if len(response_text) > 200 else response_text
        })

    avg_latency = total_time / len(questions)
    hit_rate = hit_count / len(questions)
    
    print(f"\n--- 基准测试完成 ---")
    print(f"平均响应时间: {avg_latency:.4f} 秒")
    print(f"命中率 (Hit Rate): {hit_rate:.2%}")
    
    return pd.DataFrame(results)

# --- 3. 运行测试 ---
if client and retriever and engine:
    print("\n--- 运行混合搜索系统基准测试 ---")
    results_df = run_benchmark(eval_questions)
    display(results_df)
else:
    print("⚠️ 系统未完全初始化，无法运行基准测试。")


--- 运行混合搜索系统基准测试 ---
🔍 正在进行向量检索...
  - RAG 上下文获取完毕。
🔍 正在生成并执行 SQL 查询...
  - 生成的 SQL: 
SELECT course.title  
FROM course  
JOIN teaches ON course.course_id = teaches.course_id  
JOIN instructor ON teaches.ID = instructor.ID  
WHERE instructor.name = 'Romero';
  - SQL 结果: 
查询成功！共找到 4 条记录：
                      title
           Image Processing
           Image Processing
International Communication
                 Journalism

🤖 正在综合所有信息生成最终答案...
🔍 正在进行向量检索...
  - RAG 上下文获取完毕。
🔍 正在生成并执行 SQL 查询...
  - 生成的 SQL: 
SELECT i.name
FROM instructor i
JOIN teaches t ON i.ID = t.ID
JOIN course c ON t.course_id = c.course_id
WHERE c.title = 'Manufacturing';
  - SQL 结果: 
查询成功！共找到 1 条记录：
  name
Mingoz

🤖 正在综合所有信息生成最终答案...
🔍 正在进行向量检索...
  - RAG 上下文获取完毕。
🔍 正在生成并执行 SQL 查询...
  - 生成的 SQL: 
SELECT * FROM course WHERE title = '学术诚信';
  - SQL 结果: 
查询结果为空，数据库中没有匹配的记录。

🤖 正在综合所有信息生成最终答案...
🔍 正在进行向量检索...
  - RAG 上下文获取完毕。
🔍 正在生成并执行 SQL 查询...
  - 生成的 SQL: 
SELECT course.credits
FROM course
JOIN teaches ON course

,question,latency,is_hit,response
0,查询主讲老师为 'Romero' 的课程名称是什么？,1.290928,True,查询到主讲老师为 'Romero' 的课程名称是：Image Processing。
1,Manufacturing 这门课程的授课老师是谁？,1.188208,True,Manufacturing 这门课程的授课老师是 Mingoz。
2,关于学术诚信有什么规定？,1.098287,True,根据提供的信息，教学政策中提到所有学生必须遵守学术诚信原则，严禁抄袭。关于更详细的学术诚信规...
3,查询 'Lembr' 老师教授的课程学分是多少？,1.172883,True,'Lembr' 老师教授的课程学分是 4.0。
